In [ ]:
from __future__ import annotations

import glob
import os
import time

from stable_baselines3 import PPO
from stable_baselines3.ppo import CnnPolicy, MlpPolicy

from pettingzoo.test import api_test
from lib.briscola_env.briscola_env import BriscolaEnv

In [ ]:
env = BriscolaEnv()

api_test(env, num_cycles=1_000_000)
